In [1]:
import pickle
import joblib
import sklearn
import random
import os
import pandas as pd
import numpy as np

from src.pipelines.magic_loop import load_transformation, filter_drop, \
                                      transformation_pipeline, train_test_split, NUM_VARS, CAT_VARS

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
sklearn.__version__

'0.23.2'

In [3]:
os.chdir("..")

'/home/carotova/Documents/MCD/Intro_DS/accidentes-viales-CDMX'

In [32]:
path = os.getcwd()
path

'/home/carotova/Documents/MCD/Intro_DS/accidentes-viales-CDMX'

In [4]:
joblib.__version__

'0.16.0'

In [33]:
df = load_transformation(path)

Opening feature engineering pickle from output path
Feature Engineering pickle successfully retrieved.


In [6]:
path = "output/random_forest_Leo.joblib"
model = joblib.load(path)

In [7]:
model

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             estimator=RandomForestClassifier(oob_score=1, random_state=1789),
             n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15],
                         'n_estimators': [100, 200, 400]},
             scoring='precision')

In [34]:
df = filter_drop(df)

Dropping columns and Nan's (don't worry, it'll be ok)


In [59]:
df['incidente_c4'] = df['incidente_c4'].str.lower()

In [69]:
X_train, y_train, X_test, y_test = train_test_split(df)

Performing train test split
Train test split successfully performed


In [70]:
set(X_train['incidente_c4'].unique())

{'accidente-choque con lesionados',
 'accidente-choque con prensados',
 'accidente-choque sin lesionados',
 'accidente-ferroviario',
 'accidente-motociclista',
 'accidente-otros',
 'accidente-persona atrapada / desbarrancada',
 'accidente-vehiculo atrapado',
 'accidente-vehiculo desbarrancado',
 'accidente-volcadura',
 'cadáver-accidente automovilístico',
 'cadáver-atropellado',
 'detención ciudadana-accidente automovilístico',
 'detención ciudadana-atropellado',
 'lesionado-accidente automovilístico',
 'lesionado-atropellado',
 'sismo-choque con lesionados',
 'sismo-choque con prensados',
 'sismo-choque sin lesionados',
 'sismo-persona atropellada'}

In [71]:
set(X_train['incidente_c4'].unique()) - set(X_test['incidente_c4'].unique())

{'lesionado-accidente automovilístico'}

In [72]:
set(X_test['incidente_c4'].unique()) - set(X_train['incidente_c4'].unique())

{'accidente-ciclista',
 'accidente-monopatín',
 'accidente-vehículo atrapado-varado',
 'mi ciudad-calle-incidente de tránsito',
 'mi ciudad-taxi-incidente de tránsito'}

In [67]:
# 
# 'mi ciudad-calle-incidente de tránsito',
# 'mi ciudad-taxi-incidente de tránsito',
set(X_test['incidente_c4'].unique())

{'accidente-choque con lesionados',
 'accidente-choque con prensados',
 'accidente-choque sin lesionados',
 'accidente-ciclista',
 'accidente-ferroviario',
 'accidente-monopatín',
 'accidente-motociclista',
 'accidente-otros',
 'accidente-persona atrapada / desbarrancada',
 'accidente-vehiculo atrapado',
 'accidente-vehiculo desbarrancado',
 'accidente-vehículo atrapado-varado',
 'accidente-volcadura',
 'cadáver-accidente automovilístico',
 'cadáver-atropellado',
 'detención ciudadana-accidente automovilístico',
 'detención ciudadana-atropellado',
 'lesionado-atropellado',
 'mi ciudad-calle-incidente de tránsito',
 'mi ciudad-taxi-incidente de tránsito',
 'sismo-choque con lesionados',
 'sismo-choque con prensados',
 'sismo-choque sin lesionados',
 'sismo-persona atropellada'}

In [24]:
X_train = transformation_pipeline(X_train, NUM_VARS, CAT_VARS)

Features are transformed and ready for magic loop. What about you?


In [25]:
X_test = transformation_pipeline(X_test, NUM_VARS, CAT_VARS)

Features are transformed and ready for magic loop. What about you?


In [26]:
X_train.shape

(967884, 31)

In [27]:
X_test.shape

(414808, 36)

In [13]:
# predicciones con el mejor predictor 
predicted_labels = model.predict(X_test)

ValueError: Number of features of the model must match the input. Model n_features is 31 and input n_features is 36 

## ROC

In [12]:
from sklearn.metrics import roc_curve, roc_auc_score

import matplotlib.pyplot as plt

%matplotlib inline

fpr, tpr, thresholds = roc_curve(y_test, predicted_scores[:,1], pos_label=1)

plt.clf()
plt.plot([0,1],[0,1], 'k--', c="red")
plt.plot(fpr, tpr)
plt.title("ROC best RF, AUC: {}".format(roc_auc_score(y_test, predicted_labels)))
plt.xlabel("fpr")
plt.ylabel("tpr")
plt.show()

NameError: name 'predicted_scores' is not defined